In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import load_img
from keras.utils import img_to_array
from keras.models import load_model
from keras.metrics import top_k_categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
FOOD_PATH = "food/"
FOOD_DIR = glob.glob("food/*")
CATEGORIES_PATH = "categories.csv"

MODELS = ["food-seg-103-xception", "food-seg-103-densenet201", "food-seg-103-inceptionresnetv2"]
CHECKPOINT_PATHS = ["checkpoints/" + MODEL + ".h5" for MODEL in MODELS]
MODEL_PATHS = ["models/" + MODEL + ".h5" for MODEL in MODELS]

IMAGE_SIZE = 512
TOP_N = 5

In [ ]:
categories = list(pd.read_csv(CATEGORIES_PATH, header=None)[0])

In [ ]:
def acc_top5(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [ ]:
def load_models():
    loaded_models = []
    for CHECKPOINT_PATH in CHECKPOINT_PATHS:
        loaded_models.append(load_model(CHECKPOINT_PATH, custom_objects={"acc_top5": acc_top5}))
    print("Models Loaded")
    return loaded_models

In [ ]:
models = load_models()

In [ ]:
def predict(model, filepath, top_n=1):
    test_image = load_img(filepath, target_size=(IMAGE_SIZE, IMAGE_SIZE))
    test_image_array = img_to_array(test_image)
    test_image_array = np.expand_dims(test_image_array, axis=0)
    test_image_array = test_image_array / 255.
    prediction = model.predict(test_image_array, verbose=0)
    predicted_label = np.argsort(prediction[0])[::-1][:top_n]
    predicted_score = prediction[0][predicted_label]
    return predicted_label, predicted_score

In [ ]:
def ensemble_predict(filepath, top_n=1):
    predicted_labels = []
    predicted_scores = []
    prediction = {}
    for model in models:
        predicted_label, predicted_score = predict(model, filepath, top_n)
        predicted_labels = predicted_labels + list(predicted_label)
        predicted_scores = predicted_scores + list(predicted_score)
        for i, label in enumerate(predicted_labels):
            if predicted_labels[i] in prediction:
                if prediction[categories[predicted_labels[i]]] < predicted_scores[i]:
                    prediction[categories[predicted_labels[i]]] = predicted_scores[i]
            else:
                prediction[categories[predicted_labels[i]]] = predicted_scores[i]
    prediction = dict(sorted(prediction.items(), key=lambda item: item[1], reverse=True)[:top_n])
    return prediction

In [ ]:
plt.figure(figsize=(10, 3))

for idx, food in enumerate(FOOD_DIR):
    pred = ensemble_predict(food, TOP_N)
    plt.subplot(1, len(FOOD_DIR), idx + 1)
    img = plt.imread(food)
    plt.title(list(pred.keys())[0])
    plt.imshow(img, cmap="gray")
    plt.axis("off")
    print("=" * 50)
    print(list(pred.keys())[0])
    print(pred)
    print("=" * 50)
plt.tight_layout()